In [1]:
# Prima prova VQE con oo-PUCCD

''' ElectronicStructureProblem '''

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

LiH = "Li 0 0 0; H 0 0 0.6" 

driver = PySCFDriver(
    atom=LiH,
    basis="sto3g", # 3 gaussiane per approssimare una funzione di Slater
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
print(problem)


In [3]:
''' Mapper '''

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.mappers import ParityMapper

mapper = JordanWignerMapper()
#mapper = ParityMapper() # --> occhio che ansatz e operators devono essere mappati con lo stesso mapper

''' Ansatz '''

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator # Estimator deprecated 
from qiskit_nature.second_q.circuit.library import HartreeFock, PUCCD

# Pair unitary cluster couple double
ansatz = PUCCD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [4]:
from qiskit_algorithms.gradients import LinCombEstimatorGradient

estimator = Estimator()
gradient = LinCombEstimatorGradient(estimator) 


vqe_solver = VQE(estimator, ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

vqe_solver
''' Calculation and results '''

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

# mi fa solo buttare 10s ogni volta
res = calc.solve(problem) 
# print(res)

initial_point = res.raw_result.optimal_point
vqe_solver.initial_point = initial_point

/var/folders/88/gfgbzsfs2m759nlsp4wspv400000gp/T/ipykernel_7656/80257411.py:3: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [5]:

# groundenergy è solo la computed part
computed_part = res.groundenergy
print ('computed_part = ', computed_part)

# Nuclear repulsion energy (Hartree): 
nuclear_repulsion_energy = res.nuclear_repulsion_energy
print ('nuclear_repulsion_energy = ', nuclear_repulsion_energy)

total_energy = computed_part + nuclear_repulsion_energy
print ('total_energy = ', total_energy)



computed_part =  -9.962371545689157
nuclear_repulsion_energy =  2.6458860546
total_energy =  -7.316485491089157


## ORBITAL OPT

In [6]:
from oovqe_algorithm_mio import OrbitalOptimizationVQE as ooVQE

Per inizializzare un oggetto ooVQE servono:
1. un Estimator 'transformation'
1. un solver (vqe_solver precedente)
1. argomenti opzionali

In [7]:
vqe_solver = VQE(estimator, ansatz, SLSQP(maxiter=10), gradient=gradient) # ridefinisco stavolta con gradient
# maxiter piccolo finché sono in testing
vqe_solver.optimizer.set_max_evals_grouped(10)

## TEST COMPLETO:

In [8]:
ooVQE_solver = ooVQE(JordanWignerMapper(), vqe_solver, initial_point=None)

In [9]:
oovqe_result = ooVQE_solver.solve(driver, problem)

print(oovqe_result) # 4min 46s per una sola distanza!

The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 21.1094991911992


parameters:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1]
Parameters of wavefunction are: 
%s array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
Parameters of orbital rotation are: 
%s array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1])
orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Nature matrix a:  [[ 0.97526959  0.0787423   0.08765007  0.09718668  0.10738465  0.1182778 ]
 [-0.1182778   0.97526959  0.0787423   0.08765007  0.09718668  0.10738465]
 [-0.10738465 -0.1182778   0.97526959  0.0787423   0.08765007  0.09718668]
 [-0.09718668 -0.10738465 -0.1182778   0.97526959  0.0787423   0.08765007]
 [-0.08765007 -0.09718668 -0.10738465 -0.1182778   0.97526959  0.0787423 ]
 [-0.0787423  -0.08765007 -0.09718668 -0.10738465 -0.

The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 21.1094991911992


orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [0.10001 0.1     0.1     0.1     0.1     0.1     0.1     0.1     0.1
 0.1     0.1     0.1     0.1     0.1     0.1    ]
orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [0.09999 0.1     0.1     0.1     0.1     0.1     0.1     0.1     0.1
 0.1     0.1     0.1     0.1     0.1     0.1    ]


The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 21.1094991911992
The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen f

...

orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [-0.09110015  0.00447054  0.00136203  0.00928039 -0.10734545  0.11925808
  0.12413339  0.00728634 -0.0495029   0.08667142  0.04412954  0.060259
  0.04180319  0.06074912 -0.03269113]
orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [-0.09110015  0.00447054  0.00136203  0.00928039 -0.10734545  0.11925808
  0.12413339  0.00728634 -0.0495029   0.08667142  0.04412954  0.060259
  0.04180319  0.06072912 -0.03269113]


The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 21.1094991911992
The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen f

orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [-0.09110015  0.00447054  0.00136203  0.00928039 -0.10734545  0.11925808
  0.12413339  0.00728634 -0.0495029   0.08667142  0.04412954  0.060259
  0.04180319  0.06073912 -0.03268113]
orbital_rotations:  15 
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]
self._parameters:  15 
 [-0.09110015  0.00447054  0.00136203  0.00928039 -0.10734545  0.11925808
  0.12413339  0.00728634 -0.0495029   0.08667142  0.04412954  0.060259
  0.04180319  0.06073912 -0.03270113]


The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 21.1094991911992


result:  {   'aux_operator_eigenvalues': {   'AngularMomentum': (np.float64(0.0), {}),
                                    'Magnetization': (np.float64(0.0), {}),
                                    'ParticleNumber': (   np.float64(3.999999999999912),
                                                          {}),
                                    'XDipole': (np.float64(0.0), {}),
                                    'YDipole': (np.float64(0.0), {}),
                                    'ZDipole': (   np.float64(3.2071328461409556),
                                                   {})},
    'aux_operators_evaluated': None,
    'cost_function_evals': 25,
    'eigenvalue': np.complex128(-10.50378078881403+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(t[5]): np.float64(-0.08369363153872576),
                              ParameterVectorElement(t[2]): np.float64(-0.000531105278839553),
                              ParameterVectorElement(t[3]): np.

In [10]:
total = oovqe_result.groundenergy + oovqe_result.nuclear_repulsion_energy 
print ('1: ', total)


1:  -7.85789473421403
